In [215]:
import pandas as pd
import numpy as np
import operator

In [216]:
submission_file_list = ['submissions/reverse_count_weight_canary_word_count_with_tag_2048_no_caption_0.40498.csv',
                        'submissions/0.352_plsr_800_pool5_to_bow_all_pca_2048.csv',
                        'submissions/0.31_submission_plsr_800_pool5_to_word2vec_n_v_ajd_300.csv',
                        'submissions/PLSR_100_BOW_all_pca_2048_tags_80_cos_0.26727.csv',
                        'submissions/MLPR_image_1000_des_BOW_all_23.csv',
                        'submissions/PLSR_256_concat_1000_1000iter_cos_0.19557.csv',
                        'submissions/submission_v2_013_0.13756.csv',
                        'submissions/test_submission_new_0.24732.csv',
                        'submissions/reverse_with_captions_in_desc_0.31181.csv']

In [217]:
submission_weight = [0.40498, 0.352, 0.31, 0.26727, 0.23, 0.19557, 0.13756, 0.24732, 0.31181]

# submission_weight = [1] * len(submission_file_list)

output_name = 'submissions/ensembled_0.40498_0.352_0.31_0.26727_0.23_0.19557_0.13756_0.24732_0.31181_score_weight.csv'

In [218]:
def calc_single(pred_list):
    score_dict = {}
    for i in range(len(pred_list)):
        for j in range(len(pred_list[i])):
            if pred_list[i][j] not in score_dict:
                score_dict[pred_list[i][j]] = 0.0
            score_dict[pred_list[i][j]] += float(len(pred_list[i]) - j) / len(pred_list[i]) * submission_weight[i]
    sorted_score = sorted(score_dict.items(), key=operator.itemgetter(1), reverse=True)
    return map(lambda x: x[0], sorted_score[:20])

In [219]:
def ensemble():
    res = []
    dfs = [pd.read_csv(file_path, encoding='utf-8') for file_path in submission_file_list]
    for i in range(dfs[0].shape[0]):
        pred_list = [df.iloc[i]['Top_20_Image_IDs'].split(' ') for df in dfs]
        res.append(' '.join(calc_single(pred_list)))
    print len(res)
    return pd.DataFrame({'Descritpion_ID': dfs[0]['Descritpion_ID'].tolist(), 'Top_20_Image_IDs': res})

In [220]:
df_ensembled = ensemble()

2000


In [221]:
print df_ensembled

     Descritpion_ID                                   Top_20_Image_IDs
0             0.txt  124.jpg 93.jpg 292.jpg 1105.jpg 1131.jpg 942.j...
1             1.txt  833.jpg 589.jpg 1742.jpg 80.jpg 818.jpg 457.jp...
2             2.txt  838.jpg 159.jpg 1724.jpg 598.jpg 445.jpg 1840....
3             3.txt  1765.jpg 1514.jpg 770.jpg 26.jpg 1215.jpg 1035...
4             4.txt  1231.jpg 742.jpg 292.jpg 942.jpg 1816.jpg 922....
5             5.txt  771.jpg 51.jpg 492.jpg 1335.jpg 1693.jpg 846.j...
6             6.txt  1216.jpg 1855.jpg 1815.jpg 1660.jpg 429.jpg 13...
7             7.txt  987.jpg 409.jpg 448.jpg 1285.jpg 689.jpg 685.j...
8             8.txt  419.jpg 719.jpg 531.jpg 103.jpg 727.jpg 1283.j...
9             9.txt  614.jpg 554.jpg 549.jpg 770.jpg 1795.jpg 1335....
10           10.txt  1421.jpg 904.jpg 1471.jpg 1665.jpg 1840.jpg 14...
11           11.txt  814.jpg 1693.jpg 492.jpg 51.jpg 771.jpg 1426.j...
12           12.txt  1653.jpg 1031.jpg 1171.jpg 1656.jpg 464.jpg 40...
13    

In [222]:
df_ensembled.to_csv(output_name, index=False)